In [6]:
pip install ortools

     |████████████████████████████████| 14.0MB 319kB/s 
     |████████████████████████████████| 1.0MB 37.8MB/s 
     |████████████████████████████████| 133kB 50.4MB/s 
ERROR: tensorflow-metadata 0.26.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0


# Single Knapsack

In [ ]:
from ortools.algorithms import pywrapknapsack_solver

In [ ]:
values = [1,2,3,4]
weights = [[1,2,3,4]]
capacities = [5]

In [ ]:
solver = pywrapknapsack_solver.KnapsackSolver(pywrapknapsack_solver.KnapsackSolver.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')
solver.Init(values, weights, capacities)
computed_value = solver.Solve()

In [ ]:
print(f"total value: {computed_value}\n")

total value: 5



In [ ]:
print("indexes: ", end="")
for i in range(len(values)):
  if solver.BestSolutionContains(i):  # BestSolutionConatins checks if the index is in the knapsack
    print(i, end=", ")
print("\b\b")

indexes: 1, 2, 


In [ ]:
[i for i in solver.__dir__() if "__" not in i]

['this',
 'thisown',
 'KNAPSACK_BRUTE_FORCE_SOLVER',
 'KNAPSACK_64ITEMS_SOLVER',
 'KNAPSACK_DYNAMIC_PROGRAMMING_SOLVER',
 'KNAPSACK_MULTIDIMENSION_CBC_MIP_SOLVER',
 'KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER',
 'KNAPSACK_MULTIDIMENSION_SCIP_MIP_SOLVER',
 'Init',
 'Solve',
 'BestSolutionContains',
 'set_use_reduction',
 'set_time_limit']

# Multiple Knapsack

In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
values = [1,2,3,4]
weights = [1,2,3,4]
capacities = [5, 3]

In [ ]:
solver = pywraplp.Solver.CreateSolver("SCIP")

x = [[0 for i in range(len(capacities))] for i in range(len(values))]
for i in range(len(weights)):
  for j in range(len(capacities)):
    x[i][j] = solver.IntVar(0, 1, name=f"x_{i}_{j}")  # create a binary variable for each element in array

for i in range(len(weights)):
  solver.Add(sum(x[i][j] for j in range(len(capacities))) <= 1)  # force each element to be in only 1 bin

for j in range(len(capacities)):
  solver.Add(sum(x[i][j] * weights[i] for i in range(len(weights))) <= capacities[j])  # force bin contents to be under capacity

In [ ]:
objective = solver.Objective()
for i in range(len(weights)):
  for j in range(len(capacities)):
    objective.SetCoefficient(x[i][j], values[i])  # set value for each row to weight by
objective.SetMaximization()

status = solver.Solve()

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
  print(f"Total packed value: {objective.Value()}")
  for i in range(len(weights)):
    for j in range(len(capacities)):
      print(x[i][j].solution_value(), end=", ")
    print("\b\b")
else:
  print("NO")

Total packed value: 8.0
1.0, 0.0, 
0.0, 0.0, 
0.0, 1.0, 
1.0, 0.0, 


Cols are bins. Rows are items

# Bin Packing

In [22]:
from ortools.linear_solver import pywraplp

In [23]:
values = [1,2,3,4]
weights = [1,2,3,4]
capacities = [1,2,3,4,5,6,7]

In [24]:
solver = pywraplp.Solver.CreateSolver("SCIP")
used = [solver.IntVar(0, 1, f"y[{j}]") for j in range(len(capacities))]

x = [[0 for i in range(len(capacities))] for i in range(len(values))]
for i in range(len(weights)):
  for j in range(len(capacities)):
    x[i][j] = solver.IntVar(0, 1, name=f"x_{i}_{j}")  # create a binary variable for each element in array

for i in range(len(weights)):
  solver.Add(sum(x[i][j] for j in range(len(capacities))) == 1)  # force each element to be in exactly 1 bin

for j in range(len(capacities)):
  solver.Add(sum(x[i][j] * weights[i] for i in range(len(weights))) <= capacities[j] * used[j])  # force bin contents to be under capacity ONLY if used

In [25]:
solver.Minimize(solver.Sum([used[j] for j in range(len(capacities))]))
status = solver.Solve()

In [27]:
if status == pywraplp.Solver.OPTIMAL:
  print(", ".join([str(int(used[i].solution_value())) for i in range(len(capacities))]))
  for i in range(len(weights)):
    for j in range(len(capacities)):
      print(int(x[i][j].solution_value()), end=", ")
    print("\b\b")
else:
  print("NO")

0, 0, 1, 0, 0, 0, 1
0, 0, 1, 0, 0, 0, 0, 
0, 0, 1, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 0, 1, 
0, 0, 0, 0, 0, 0, 1, 


Cols are bins. Rows are items. Top row is which bins are used